In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
from transformers import BertTokenizer
from IPython.display import clear_output

In [3]:
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
   AutoModel,
)


tokenizer = BertTokenizerFast.from_pretrained(PRETRAINED_MODEL_NAME)


In [4]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [5]:
import pandas as pd

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
df_train=pd.read_csv("basic8_data_replace2.csv")
df_val=pd.read_csv("basic8_val.csv")
df_total=pd.read_csv("basic8_total.csv")
len_=len(df_train)
movies=['寄生上流', '血觀音', '沙丘','返校','刻在你心底的名字','一級玩家','我的少女時代','天能']
print("data len=",len_)
print("val len=",len(df_val))

data len= 3513
val len= 350


In [8]:
#-------把太長的去掉------
MAX_LENGTH=80

df_train = df_train[~(df_train.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_val = df_val[~(df_val.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_total = df_val[~(df_total.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
#df_val_replace=df_val_replace[~(df_val_replace.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
len_=len(df_train)
print("after len=",len_)
print("after len=",len(df_val))

after len= 2629
after len= 303


C:\Users\user\AppData\Local\Temp\ipykernel_2916\3786618029.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_total = df_val[~(df_total.sentence.apply(lambda x : len(x)> MAX_LENGTH))]


In [9]:
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用

print("訓練樣本數：", len(df_train))
print(df_train.head())
print(df_val.head())

訓練樣本數： 2629
                                            sentence label
0                          網誌版，拖了好久，終於找到時間看這部獲獎無數的電影  寄生上流
1                  故事情節懸疑緊湊，每個鏡頭、片段、，對話都精心設計與安排，名不虛傳  寄生上流
2                而就像一杯美酒，好的藝術品除了本身的美感，更重，要的是品嘗過後那股後勁  寄生上流
3  寄生上流是一部後勁非常強的電影，讓我們在關注完情節之後，，想到的是更多導演可能沒有明說，卻逼...  寄生上流
4                      ，電影中最有名的台詞之一是媽媽說的不是『雖然有錢，卻很善良  寄生上流
                                            sentence label
0  金氏一家四口居於在首爾公寓的破舊半地下室中，全家因長期失業無正職工作而生活困頓，只能靠摺披薩...  寄生上流
4             自此，金氏一家四口靠著裙帶關係及陰謀詭計，全數成功進佔朴家四份穩定的高薪肥缺  寄生上流
5  數日後，朴家為慶祝多頌生日闔府外出露營度假，並託新管家忠淑留守看管，金家四口當晚於是暫駐大宅...  寄生上流
6             然而雯光在雨夜忽然到訪門前按鈴，表示因走得匆忙而忘了取回地下室遺留的重要東西  寄生上流
8                      此時突然天降豪雨，朴家被逼取消行程提前折返，令金家計畫失控  寄生上流


In [10]:
df_train.label.value_counts() / len(df_train)
df_train.to_csv("train.tsv", sep="\t", index=False)
df_train.label.value_counts() / len(df_train)

寄生上流        0.151769
沙丘          0.150247
血觀音         0.147965
返校          0.138836
刻在你心底的名字    0.133891
一級玩家        0.107265
我的少女時代      0.089768
天能          0.080259
Name: label, dtype: float64

In [11]:
df_val.to_csv("val.tsv", sep="\t", index=False)
df_val.label.value_counts() / len(df_val)

血觀音         0.303630
天能          0.244224
我的少女時代      0.105611
沙丘          0.079208
寄生上流        0.072607
刻在你心底的名字    0.072607
返校          0.062706
一級玩家        0.059406
Name: label, dtype: float64

In [12]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_val.label.value_counts() / len(df_val)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index
            df_val = df_val[~df_val.index.isin(bad_label_index)]
    percentage=df_val.label.value_counts() / len(df_val)
print(percentage)


df_val.to_csv("val.tsv", sep="\t", index=False)
"""

'\npercentage=df_val.label.value_counts() / len(df_val)\nwhile max(percentage)>0.20:\n    for serie in series:\n        if percentage[serie]>0.20:\n            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index\n            df_val = df_val[~df_val.index.isin(bad_label_index)]\n    percentage=df_val.label.value_counts() / len(df_val)\nprint(percentage)\n\n\ndf_val.to_csv("val.tsv", sep="\t", index=False)\n'

In [13]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_total.label.value_counts() / len(df_total)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_total[df_total["label"]==serie].sample(len(df_total)//20).index
            df_total = df_total[~df_total.index.isin(bad_label_index)]
    percentage=df_total.label.value_counts() / len(df_total)
print(percentage)
"""
df_total.to_csv("total.tsv", sep="\t", index=False)

tokens_tensor：代表識別每個 token 的索引值，用 tokenizer 轉換即可  

segments_tensor：用來識別句子界限。第一句所有index對應 0，第二句所有index對應 1。另外注意句子間的 [SEP] 為 0  

masks_tensor：用來界定自注意力機制範圍。1 讓 BERT 關注該位置，0 則代表是 padding 不需關注  //因為我們會zero padding讓batch裡大小一致


In [14]:
label_map = {movie:i  for i, movie in enumerate(movies)}
inv_label_map = {v: k for k, v in label_map.items()}
print(inv_label_map)


def indices_to_labels(indices):
    for i in range(len(indices)):
        indices[i] = inv_label_map[indices[i]]
    return indices


print(indices_to_labels([2, 3, 4]))

{0: '寄生上流', 1: '血觀音', 2: '沙丘', 3: '返校', 4: '刻在你心底的名字', 5: '一級玩家', 6: '我的少女時代', 7: '天能'}
['沙丘', '返校', '刻在你心底的名字']


In [15]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset

#https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
class SeriesDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer,label_map):
        #assert mode in ["train", "test","val"]  # 確定檔案名稱只接受train或test
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")   #讀處裡好的tsv避免跟所有檔案的csv搞混
        self.len = len(self.df)
        self.label_map = label_map
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer

    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        """
        if self.mode == "test":
            text, label = self.df.iloc[idx, :].values
            label_tensor = None
        else:
        """
        text,label = self.df.iloc[idx, :].values
        # 將 label 文字也轉換成索引方便轉換成 tensor
        label_id = self.label_map[label]
        label_tensor = torch.tensor(label_id)

        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        return (tokens_tensor, label_tensor)

    def __len__(self):
        return self.len

In [16]:
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = SeriesDataset("train", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
valset=SeriesDataset("val", tokenizer=tokenizer,label_map=label_map)
totalset=SeriesDataset("total", tokenizer=tokenizer,label_map=label_map)
#valset_replace = SeriesDataset("val_replace", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
print(trainset.df)
print(valset.df)
#print(valset_replace.df)

                                               sentence label
0                             網誌版，拖了好久，終於找到時間看這部獲獎無數的電影  寄生上流
1                     故事情節懸疑緊湊，每個鏡頭、片段、，對話都精心設計與安排，名不虛傳  寄生上流
2                   而就像一杯美酒，好的藝術品除了本身的美感，更重，要的是品嘗過後那股後勁  寄生上流
3     寄生上流是一部後勁非常強的電影，讓我們在關注完情節之後，，想到的是更多導演可能沒有明說，卻逼...  寄生上流
4                         ，電影中最有名的台詞之一是媽媽說的不是『雖然有錢，卻很善良  寄生上流
...                                                 ...   ...
2624  ，那意味著這是一種雞生蛋、蛋生雞的問題，你所觀察到的整段時間長度只能是濃縮成一個，點，才可能...    天能
2625  ，但在同時發生的狀況下，必然不可能逆行時間，因為一旦逆行，意味著你跟其他事件已經，脫鉤，不符...    天能
2626  ，那要能逆行時間，你只能破壞掉命定論，若是破壞線性時間，因為本作的命定論並非用收，束來解釋，...    天能
2627              ，所以要同時成立本作三個設定，命定論 + 線性時間 +可逆行時間必然不成立    天能
2628                               ，所以還是不要在意這部的時空理論吧‧‧‧    天能

[2629 rows x 2 columns]
                                              sentence label
0    金氏一家四口居於在首爾公寓的破舊半地下室中，全家因長期失業無正職工作而生活困頓，只能靠摺披薩...  寄生上流
1               自此，金氏一家四口靠著裙帶關係及陰謀詭計，全數成功進佔朴家四份穩定的高薪肥缺  寄生上流
2    數日後，朴家為慶祝多頌生日闔府外出露營度假，並託新管家忠淑留守看管，金家四口當晚於是暫

In [17]:
import random
# 選擇一個樣本
sample_idx = int(random.random()*len(trainset))
print(sample_idx)
# 將原始文本拿出做比較
text,label = trainset.df.iloc[sample_idx]

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor,label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

label_tensor   ：{label_tensor}
--------------------

[還原 tokens_tensors]
{combined_text}
""")

2371
[原始文本]
句子 1：即便遍，體麟傷，你的對抗直來直往，坦坦蕩蕩
分類  ：我的少女時代

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 1315,  912, 6881, 8024, 7768, 7929, 1003, 8024,  872, 4638, 2205,
        2834, 4684,  889, 4684, 2518, 8024, 1788, 1788, 5939, 5939,  102])

label_tensor   ：6
--------------------

[還原 tokens_tensors]
[CLS]即便遍，體麟傷，你的對抗直來直往，坦坦蕩蕩[SEP]



In [18]:
print(type(trainset[0]))
print(len(trainset))
print(type(trainset))

#trainset 直接用[]來看是一個(2657,)個tuple of three tensor

<class 'tuple'>
2629
<class '__main__.SeriesDataset'>


In [19]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `MovieDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list由前面的3個tenors組成，裡頭的每個 element 都是
# 剛剛定義的 `MovieDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]       #把每個sample的tensor取出來放進list 產生一個 list of tensor (batch_size,)
    
    # 測試集有 labels
    if samples[0][1] is not None:
        label_ids = torch.stack([s[1] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度(要同時做好幾個RNN就要batch中每個元素一樣長)
    #https://blog.csdn.net/qq_43391414/article/details/123289492
    
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    
    return tokens_tensors, label_ids


In [20]:
#把data跟寫好的create_mini_batch放進trainloader
BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )

valloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
totalloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""
valloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""

'\nvalloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,\n                         shuffle=True,\n                         collate_fn=create_mini_batch\n                         )\n'

In [21]:
data = next(iter(trainloader))  # 取出traing loader 裡的一個mini_batch

tokens_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([32, 82]) 
tensor([[ 101, 7440, 3152,  ...,    0,    0,    0],
        [ 101, 8024, 1762,  ..., 1377,  809,  102],
        [ 101, 8024, 2769,  ...,    0,    0,    0],
        ...,
        [ 101,  852, 3291,  ...,    0,    0,    0],
        [ 101, 8024, 1008,  ...,    0,    0,    0],
        [ 101, 4511, 3301,  ...,    0,    0,    0]])
------------------------
label_ids.shape        = torch.Size([32])
tensor([3, 3, 4, 3, 4, 3, 4, 3, 4, 2, 2, 4, 1, 5, 0, 4, 2, 3, 5, 5, 4, 5, 7, 3,
        4, 0, 0, 1, 1, 0, 5, 1])



建立 BERT 用的 mini-batch 時最需要注意的就是 zero padding 的存在了。  
你可以發現除了 lable_ids 以外，其他 3 個 tensors 的每個樣本的最後大都為 0，  
這是因為每個樣本的 tokens 序列基本上長度都會不同，需要補 padding。

In [22]:
from transformers import BertForSequenceClassification

In [48]:
import os
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"
NUM_LABELS = len(label_map)
PATH="model_sam80.pt"

model = BertForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS,attention_probs_dropout_prob=0.10,
        hidden_dropout_prob=0.10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
#uncommend to load model

if os.path.isfile(PATH):
    print("=====>LOAD checkpoint")
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    print("======>LOAD new model")


Some weights of the model checkpoint at ckiplab/bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-c

=====>LOAD checkpoint


KeyError: 'step'

In [24]:
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=8, bias=True)


In [25]:
""""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        wrong_list=[]
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:     #檢查model是否在GPU
                data = [t.to("cuda:0") for t in data if t is not None]    #sentence 跟label tensor從tuple改放到cuda list
            tokens_tensors= data[0]   #data 已經放在cuda了 如果沒有的話 應該會報錯
            outputs = model(input_ids=tokens_tensors)
            #ouputs 原本是 SequenceClassifierOutput class
            #用outputs[0]可以取出他的tensor部分 shape=(64,3)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
 
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[1]
                total += labels.size(0)
                correct += (pred == labels).sum().item()   #.item把tensor 轉為純量
                
                cur=0
                for i in (pred!=labels):
                    if i==True:
                        wrong_list.append((tokens_tensors[cur],pred[cur].item(),labels[cur].item()))
                    cur+=1
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))  #最後會形成一長串所有的prediction
    
    if compute_acc:
        acc = correct / total
        return wrong_list, acc
    return predictions
    

In [49]:

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)     #把model移到devive 應該會是cuda:0
acc_record =0.5 #set 0 to start saving model
train_acc_list=[]
val_acc_list=[]

device: cuda:0


In [50]:

_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)
#print("total prediction:",_.shape)


classification acc: 0.9539748953974896


In [28]:
#for data in trainloader:
    #print(data[0].shape)
print(next(model.parameters()).shape)

torch.Size([21128, 768])


In [29]:
data = next(iter(trainloader))    #並沒有紀錄iter 所以不會一直next下去 只會取trainloader的開頭 並取得next
for t in data:
    print(t.shape)
tmp=[t.to("cuda:0") for t in data if t is not None]
print(type(tmp))
print(type(data))

torch.Size([32, 77])
torch.Size([32])
<class 'list'>
<class 'tuple'>


In [30]:
from sam import SAM

from utility.smooth_cross_entropy import smooth_crossentropy
from utility.initialize import initialize
from utility.step_lr import StepLR
from utility.bypass_bn import enable_running_stats, disable_running_stats

输入：
    input_ids：训练集，torch.LongTensor类型，shape是[batch_size, sequence_length]
    token_type_ids：可选项，当训练集是两句话时才有的。
    attention_mask：可选项，当使用mask才有，可参考原论文。
    labels：数据标签，torch.LongTensor类型，shape是[batch_size]

In [51]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.00003, momentum=0.9)
#optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.001, momentum=0.9)
EPOCHS = 15
FREQ=10
#acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(trainloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # first forward-backward step
        enable_running_stats(model)
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        
        loss = smooth_crossentropy(logits, labels)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)     
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        smooth_crossentropy(logits, labels).mean().backward()
        optimizer.second_step(zero_grad=True)
        
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))

        # 紀錄當前 batch loss
        running_loss += loss.mean().item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
        train_acc_list.append(acc)
        val_acc_list.append(val_acc)
        print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
              (epoch + 1, running_loss, acc, val_acc))
        if val_acc > acc_record:
            acc_record = val_acc
            SAVE_PATH = "model_sam"+str(int(val_acc*100))+".pt"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, SAVE_PATH)
            print("Save Checkpoint to:", SAVE_PATH)


epoch=0 iteration=0 acc=0.8125
epoch=0 iteration=10 acc=0.71875
epoch=0 iteration=20 acc=0.875
epoch=0 iteration=30 acc=0.78125
epoch=0 iteration=40 acc=0.8125
epoch=0 iteration=50 acc=0.84375
epoch=0 iteration=60 acc=0.84375
epoch=0 iteration=70 acc=0.84375
epoch=0 iteration=80 acc=0.90625
[epoch 1] loss: 18.587, acc: 0.808,val acc: 0.782
Save Checkpoint to: model_sam78.pt
epoch=1 iteration=0 acc=0.59375
epoch=1 iteration=10 acc=0.71875
epoch=1 iteration=20 acc=0.6875
epoch=1 iteration=30 acc=0.90625
epoch=1 iteration=40 acc=0.75


KeyboardInterrupt: 

In [52]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.00003, momentum=0.9)
#optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.001, momentum=0.9)
EPOCHS = 1
FREQ=10
#acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(totalloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # first forward-backward step
        enable_running_stats(model)
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        
        loss = smooth_crossentropy(logits, labels)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)     
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        smooth_crossentropy(logits, labels).mean().backward()
        optimizer.second_step(zero_grad=True)
        
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))

        # 紀錄當前 batch loss
        running_loss += loss.mean().item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
    train_acc_list.append(acc)
    val_acc_list.append(val_acc)
    print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
          (epoch + 1, running_loss, acc, val_acc))
    if val_acc > acc_record:
        acc_record = val_acc
        SAVE_PATH = "model_sam"+str(int(val_acc*100))+".pt"
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, SAVE_PATH)
        print("Save Checkpoint to:", SAVE_PATH)


epoch=0 iteration=0 acc=0.375
epoch=0 iteration=10 acc=0.5
epoch=0 iteration=20 acc=0.25
epoch=0 iteration=30 acc=0.375
[epoch 1] loss: 17.168, acc: 0.498,val acc: 0.858
Save Checkpoint to: model_sam85.pt
Wall time: 14.1 s


In [33]:
#print wrong predict of val
model.eval()
wrong_list, acc = get_predictions(model, valloader, compute_acc=True)
print(acc)
print(wrong_list[0])
for wrong in wrong_list:
    tokens = tokenizer.convert_ids_to_tokens(wrong[0])
    combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string
    combined_text=combined_text.replace("[PAD]","")
    print(combined_text)
    print("predict:{} true:{}".format(inv_label_map[wrong[1]],inv_label_map[wrong[2]]))

0.9834983498349835
(tensor([ 101, 4500, 3067, 5437, 3582, 6845, 7431, 6204, 3080, 4412, 1842, 4638,
        6971, 5507, 1469, 1134, 2617, 3172, 1300, 1894, 3298, 1394, 8024, 1086,
        2823, 1168,  924, 5397, 1469, 4049, 6205, 1649, 1957, 1894, 8024, 1724,
         943,  782,  671, 6629, 1343, 5648, 4777, 4955, 2792, 6719, 5966,  102,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], device='cuda:0'), 7, 2)
[CLS]用撲翼機逃離襲擊現場的鄧肯和凱恩斯博士會合，再找到保羅和潔西嘉女士，四個人一起去舊研究所躲藏[SEP]
predict:天能 true:沙丘
[CLS]顯示兩人皆知道marc##o躲在畫室[SEP]
predict:刻在你心底的名字 true:血觀音
[CLS]主角與艾弗斯注意到洞穴天井有爆破裝置，並發現一名死亡的士兵與一扇鎖上的門，死者背包上繫著一條紅繩吊飾[SEP]
predict:一級玩家 true:天能
[CLS]一日早晨，方芮欣的母親李妙子懷疑丈夫方道勤有外遇，卻遭到方道勤家暴，情緒失控的李妙子不斷念佛，並向菩薩許下希望方道勤消失的願望[SEP]
predict:寄生上流 true:返校
[CLS]院長夫人說近期流行的buffet像是在乞討一樣，殊不知身後庭院裡其他賓客正用著buffet[SEP]
predict:寄生上流 true:血觀音


In [34]:
#plot
#plt.xlim(0,2200 ) # 設定 x 軸座標範圍
import matplotlib.pyplot as plt
plt.ylim(0, 1) # 設定 y 軸座標範圍

plt.xlabel('epoch', fontsize="10") # 設定 x 軸標題內容及大小
plt.ylabel('Accuracy', fontsize="10") # 設定 y 軸標題內容及大小
plt.title('Accuracy in use replace data (dropout=0.1)(8 movie prediction)', fontsize="18") # 設定圖表標題內容及大小

plt.plot(range(len(train_acc_list)),train_acc_list,marker='o', color='blue',label="Training Accuracy")
plt.plot(range(len(val_acc_list)),val_acc_list,marker='o', color='red',label="Validation Accuracy")
#for x,y in zip(range(len(val_acc_list)),val_acc_list): 
#    plt.text(x, y+0.01, "({}, {:.3f})".format(x,y))

plt.legend()
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.savefig("Movie Accuracy use replace data (dropout=0.1).png",dpi=100)

plt.clf()

<Figure size 1500x1000 with 0 Axes>

In [61]:
model.eval()
test_text = "[CLS]在浴室裡做色色的事[SEP]"  # 盡量湊到6個字以上 ex:xx到底是甚麼
tokens = tokenizer.tokenize(test_text)
ids = tokenizer.convert_tokens_to_ids(tokens)

id_tensor = torch.tensor([ids]).to(device)
print(test_text)
print(tokens[:])
print(ids[:])
k = 8
with torch.no_grad():
    outputs = model(input_ids=id_tensor)
    prediction = outputs[0]
    probs, indices = torch.topk(torch.softmax(prediction, -1), k)
    probs = probs.tolist()
    indices = indices.tolist()
    probs = probs[0]
    indices = indices[0]
for i, (indice, p) in enumerate(zip(indices, probs), 1):
    label = inv_label_map[indice]
    print("Top {} ({:2}%)：{}".format(i, int(p * 100), label[:10]))

[CLS]在浴室裡做色色的事[SEP]
['[CLS]', '在', '浴', '室', '裡', '做', '色', '色', '的', '事', '[SEP]']
[101, 1762, 3861, 2147, 6174, 976, 5682, 5682, 4638, 752, 102]
Top 1 (75%)：血觀音
Top 2 ( 8%)：寄生上流
Top 3 ( 7%)：我的少女時代
Top 4 ( 2%)：一級玩家
Top 5 ( 2%)：天能
Top 6 ( 1%)：刻在你心底的名字
Top 7 ( 1%)：返校
Top 8 ( 0%)：沙丘


In [36]:
def MoviePrediction(text,model,tokenizer):
    test_text="[CLS]"+text+"[SEP]"   #盡量湊到6個字以上 ex: xx到底是甚麼
    tokens = tokenizer.tokenize(test_text)
    ids = tokenizer.convert_tokens_to_ids(tokens)

    id_tensor = torch.tensor([ids]).to(device)
    print(text)
    print(ids[:])
    k=8
    with torch.no_grad():
        outputs = model(input_ids=id_tensor)     
        prediction = outputs[0]
        probs, indices = torch.topk(torch.softmax(prediction, -1), k)
        probs=probs.tolist()
        indices=indices.tolist()
        probs=probs[0]
        indices=indices[0]
    for i, (indice, p) in enumerate(zip(indices, probs), 1):
        label=inv_label_map[indice]
        print("Top {} ({:2}%)：{}".format(i, int(p * 100), label[:10]), '...')

In [37]:
#text="姐姐其實是自己的媽媽"
text="託妹妹偽造名校的學校"

MoviePrediction(text,model,tokenizer)

NameError: name 'MoviePrediction' is not defined

In [ ]:
tokenizer.convert_tokens_to_ids("_")